# Werkcollege-opdrachten Week 1.2

## Voorbereiding

Importeer in het codeblok hieronder de packages die worden gebruikt om data in te lezen. Geef er ook de gebruikelijke aliassen aan.<br>
N.B.: de 2 reeds geschreven coderegels zorgen ervoor dat eventuele warnings, die code-outputs lelijker maken, uitgezet worden.

In [ ]:
import pandas as pd
import warnings
warnings.simplefilter('ignore')

Zet de volgende bestanden in een makkelijk terug te vinden map:
- go_sales_train.sqlite
- go_crm_train.sqlite
- go_staff_train.sqlite

Bestudeer de bovenste 3 bestanden in DB Browser (SQLite), <a href="https://sqlitebrowser.org/dl/">hier</a> te downloaden. Wat valt je op qua datatypen?<br>

## Databasetabel inlezen

Creëer een databaseconnectie met het bestand go_sales_train.sqlite.

In [2]:
import sqlite3 as sql
conn = sql.connect('../data/raw/go_sales_train.sqlite')

<b>Let goed op:</b><br>
Als je per ongeluk een verkeerde bestandsnaam ingeeft, maakt Python zélf een leeg databasebestand aan! Er ontstaat dan dus een nieuwe .sqlite, en dat is nadrukkelijk <u>niet de bedoeling.</u>

Gebruik de onderstaande sql_query om te achterhalen welke databasetabellen in go_sales_train zitten.

In [5]:
sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
conn_cursor = conn.cursor()

conn_cursor.execute(sql_query)

output = conn_cursor.fetchall()

print(output)

[('country',), ('order_details',), ('order_header',), ('order_method',), ('product',), ('product_line',), ('product_type',), ('retailer_site',), ('return_reason',), ('returned_item',), ('sales_branch',), ('sales_staff',)]


Krijg je lege output? Dan heb je per ongeluk een leeg  databasebestand (.sqlite) aangemaakt.<br>
Lees de informatie onder het kopje <u>Let goed op:</u> hierboven nog eens goed door.

Gebruik de gecreëerde databaseconnectie om de resultaten van de volgende query in een DataFrame te zetten:<br>
*SELECT * FROM sales_staff* 

In [ ]:
sql_query = "SELECT * FROM sales_staff;"
conn_cursor = conn.cursor()

conn_cursor.execute(sql_query)

output = conn_cursor.fetchall()

salesStaffDf = pd.DataFrame(output)

print(salesStaffDf)

## Datumkolommen

Zoals je misschien al hebt gezien in DB Browser, zijn datums als TEXT opgeslagen, en niet als DATE, DATETIME o.i.d. Hier moeten we dus nog even "typische datumkolommen" van maken. Dat doen we met de volgende code:

In [ ]:
sales_staff['DATE_HIRED'] = pd.to_datetime(sales_staff['DATE_HIRED'])
sales_staff.dtypes

Als we hier het jaar uit willen halen, schrijven we:

In [ ]:
pd.DatetimeIndex(sales_staff['DATE_HIRED']).quarter

Deze zelfde syntax is te gebruiken voor het extraheren van kwartalen, maanden, weken en dagen. Probeer het maar eens!

## DataFrames uitsplitsen en opbouwen met Series

De volgende 5 kolommen hebben betrekking op de contactdetails van elke medewerker in dit DataFrame:
- SALES_STAFF_CODE
- WORK_PHONE
- EXTENSION
- FAX
- EMAIL

Maak van elk van deze 5 kolommen een serie.

In [ ]:
salesSerie = pd.Series(salesStaffDf.loc[:, 0])
workSerie = pd.Series(salesStaffDf.loc[:, 1])
extensionSerie = pd.Series(salesStaffDf.loc[:, 2])
faxSerie = pd.Series(salesStaffDf.loc[:, 3])
emailSerie = pd.Series(salesStaffDf.loc[:, 4])

print(salesSerie)
print(workSerie)
print(extensionSerie)
print(faxSerie)
print(emailSerie)

Zet allevijf gecreëerde series als kolommen naast elkaar in een DataFrame (*contact_details*). Gebruik pd.concat(...)<br>
Hulpvraag: welke waarde geef je aan de axis-parameter?

In [ ]:
contact_details = pd.concat([salesSerie, workSerie, extensionSerie, faxSerie, emailSerie], axis=1)

print(contact_details)

## Series en DataFrames maken vanuit lists en dictionaries

Met .head(*getal*) kan je de bovenste *getal* rijen van een tabel selecteren.<br>
Selecteer op deze manier de bovenste 5 rijen van *contact_details*.<br>
Sla dit resultaat op in een nieuw DataFrame.

In [ ]:
newData = {0: ["testcode"], 1: ["12345678"], 2: ["skhjdfkhs"], 3: ["1234"], 4: ["dhasdk@com"]}
newDf = pd.DataFrame(newData)

test = pd.concat([contact_details, newDf], ignore_index=True)
print(test.head(5))


Aan deze 10 rijen met contactdetails willen we 3 kolommen toevoegen: 'FIRST_LANGUAGE', 'SECOND_LANGUAGE' & 'THIRD_LANGUAGE'.<br>
Iedereens 'First Language' is Engels, afgekort 'EN'. Maak een lijst waarin 5x 'EN' staat.<br>
Converteer deze lijst vervolgens naar een serie en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [ ]:
firstLanguageSeries = pd.Series(["EN", "EN", "EN", "EN", "EN"], name= "FIRST_LANGUAGE")

test.insert(5, "FIRST_LANGUAGE", firstLanguageSeries)
print(test.head(5))

Maak nu de tweede kolom ('SECOND_LANGUAGE'). Maak daarvoor een dictionary, met daarin...
- Als keys: alle indexen uit het resultaat van het vorige codeblok.
- Als values: bij de eerste 3 elementen 'FR' (Frankrijk), bij de laatste 2 'DE' (Duitsland).

Maak vervolgens ook hier weer een serie van en voeg ook deze weer horizontaal samen met het rsultaat van de vorige opdracht.<br>
Vergeet niet een passende naam te geven aan de nieuw ontstane kolom.

In [ ]:
secondLanguageDictionary = {0: "FR", 1: "FR", 2: "FR", 3: "DE", 4: "DE"}

secondLanuageSeries = pd.Series(secondLanguageDictionary, name="SECOND_LANGUAGE")

test.insert(6, "SECOND_LANGUAGE", secondLanuageSeries)
print(test.head(5))

Maak ten slotte de derde kolom ('THIRD_LANGUAGE') door een dictionary te maken met daarin...
- Als key: de naam van de nieuwe kolom. Zie je het verschil qua keys met de vorige opdracht?
- Als waarde: een lijst met daarin 'NL', 'NL', 'JPN', 'JPN', 'KOR'.

Converteer deze dictionary nu naar een DataFrame en voeg deze horizontaal samen met het resultaat van de vorige opdracht.<br>
Waarom hoef je hierna de nieuw ontstane kolom geen passende naam meer te geven?

In [ ]:
thirdLanguageDictionary = {
    "THIRD_LANGUAGE": ["NL", "NL", "JPN", "JPN", "KOR"]
}

thirdLanguageDataFrame = pd.DataFrame(thirdLanguageDictionary)

test.insert(7, "THIRD_LANGUAGE", thirdLanguageDataFrame)
print(test.head(5))

## Data toevoegen

### Rijen

Gebruik de originele databasetabel *sales_staff*.<br>
Voeg een extra rij toe met eigen invulling. Zorg ervoor dat de index netjes doorloopt.<br>
Hulpvraag: welke waarde geef je nu aan axis?

In [ ]:
cursor = conn.cursor()
salesStaffQuery = "SELECT * FROM sales_staff"
response = cursor.execute(salesStaffQuery).fetchall()

column_names = [desc[0] for desc in cursor.description]

salesStaffDf2 = pd.DataFrame(response, columns=column_names)

newRow = {
    "SALES_STAFF_CODE": 125,
    "FIRST_NAME": "testname",
    "LAST_NAME": "testlastname",
    "POSITION_EN": "Branch Manager",
    "WORK_PHONE": "+31 06 49 32 64 78",
    "EXTENSION": "3453",
    "FAX": "+31 06 49 22 22 22",
    "EMAIL": "testemail@gmail.com",
    "DATE_HIRED": "01-01-2021",
    "SALES_BRANCH_CODE": 40
}

newSalesStaffDf = salesStaffDf2._append(newRow, ignore_index=True)

print(newSalesStaffDf.head(5))


### Kolommen

Voeg een kolom *FULL_NAME* toe die de waarden van *FIRST_NAME* en *LAST_NAME* achter elkaar zet, gescheiden door een spatie.

In [ ]:
firstNameColumn = newSalesStaffDf.loc[:, 'FIRST_NAME']
lastNameColumn = newSalesStaffDf.loc[:, 'LAST_NAME']

fullNameColumn = firstNameColumn + " " + lastNameColumn

# newSalesStaffDf.drop(columns=['FULL_NAME'], inplace=True)
newSalesStaffDf.insert(10, "FULL_NAME", fullNameColumn)

print(newSalesStaffDf.head(5))


## Data wijzigen

### Datatypen

Door het attribuut .dtypes van een DataFrame op te vragen krijg je een serie die per kolom het datatype weergeeft. Doe dit bij de originele databasetabel *sales_staff*

In [ ]:
print(salesStaffDf.dtypes)

Hier valt op dat elke kolom het datatype 'object' heeft: Python leest dus alles als string. Wiskundige operaties zijn hierdoor niet mogelijk.<br>
We kunnen proberen om kolommen met getallen, bijvoorbeeld de 'extension', te converteren naar een int. Zie onderstaande code.

In [ ]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'].astype(int)
sales_staff.dtypes

Dit lukt echter niet, omdat er in de kolom 'EXTENSION' lege waarden zitten die niet geconverteerd kunnen worden naar een int.<br>
Wél kan je deze naar een float converteren, zie onderstaande code:

In [ ]:
sales_staff['EXTENSION'] = sales_staff['EXTENSION'].astype(float)
sales_staff.dtypes

Als we in de rij van 'EXTENSION' kijken zien we dat de conversie van het datatype nu is gelukt.<br>
Dit is <b>randvoorwaardelijk</b> voor het uitvoeren van wiskundige operaties.<br>

### Rijen

Zorg er nu voor dat bij alle extensions 1 wordt opgeteld.

In [ ]:
newSalesStaffDf['EXTENSION'] = newSalesStaffDf['EXTENSION'] + 1

Elke 'Branch Manager' wordt nu 'General Manager'. Schrijf code zodat deze wijziging doorgevoerd wordt in het DataFrame.

In [4]:
newSalesStaffDf['POSITION_EN'] = newSalesStaffDf['POSITION_EN'].replace('Branch Manager', 'General Manager')
print(newSalesStaffDf.head(5))

NameError: name 'newSalesStaffDf' is not defined

### Kolommen

Verander de kolomnaam 'POSITION_EN' naar 'POSITION'.

In [ ]:
column_names = [desc[0] for desc in cursor.description]
newSalesStaffDf2 = pd.DataFrame(newSalesStaffDf, columns=column_names)


newSalesStaffDf2.rename(columns={'POSITION_EN': 'POSITION'}, inplace=True)
print(newSalesStaffDf2.head(5))

## Data verwijderen

### Rijen

De medewerkers op indexen 99, 100 en 101 hebben helaas ontslag genomen.<br>
Verwijder de bijbehorende rijen uit het DataFrame. Gebruik slechts één keer de .drop()-methode.

In [ ]:
newSalesStaffDf2.drop([99, 100, 101], inplace=True)
print(newSalesStaffDf2)

### Kolommen

Faxen zijn inmiddels ouderwets: niemand gebruikt zijn/haar faxnummer nog.<br>
Verwijder de bijbehorende kolom uit het DataFrame.

In [ ]:
newSalesStaffDf2.drop(columns={'FAX'}, inplace=True)
print(newSalesStaffDf2)